# Animated Aftershock Script

This is a beta script which will take a geonet quake ID and create an animated figure of the aftershock location, with size dependent on magnitude.

In [ ]:
#importing relevant modules
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
from matplotlib.animation import FFMpegWriter
from mpl_toolkits.basemap import Basemap
import os
import requests
import dateutil.parser
from datetime import timedelta, datetime
from obspy import UTCDateTime

### User input here 
Enter the quake ID, length of time in minutes and a file name, then "run all cells"

In [ ]:
firstquakeid = "2013p613797"# user input Geonet public ID 
length = 30
filename = "Grassmere"

Function follows below:

In [ ]:
def aftershock_animation(quakeid, minutes, filename):
    
    """This function takes a Geonet quakeid input and returns an animation.
    
    Data taken from the Geonet API. Some default parameters: aftershocks defined 
    as within 50 km of epicentre, map defined by .25 degrees in any direction from 
    epicentre. Presently designed for rapid maps of aftershocks, for multiple days
    would recommend changing framerate and the timelist and stime lists (the 60
    represents how many seconds per frame)"""
    
    #Queries Geonet API to retreive mainshock's data
    quake = requests.get("https://api.geonet.org.nz/quake/"+ firstquakeid).json()
    
    #Origin time from json file, in datetime object
    starttime = dateutil.parser.isoparse(quake['features'][0]['properties']['time'])
    endtime = starttime + timedelta(days = 5) #end time, 5 days after, increase if needed
    
    #strings of times for wfs query
    start, end = starttime.strftime('%Y-%m-%dT%H:%M:%S'), endtime.strftime('%Y-%m-%dT%H:%M:%S')
    #longitude/latitude for query
    lon = quake['features'][0]['geometry']['coordinates'][0]
    lat = quake['features'][0]['geometry']['coordinates'][1]
    latlon = str(lon) + "+" + str(lat)
    
    #wfs query. For further info see wfs.geonet.org.nz
    quakejson = requests.get("http://wfs.geonet.org.nz/geonet/ows?service=WFS&version=1.0.0&request="
                             "GetFeature&typeName=geonet:quake_search_v1&outputFormat=json&cql_filter="
                             "origintime>=" + start + "+AND+origintime<=" + end + "+AND+DWITHIN"
                             "(origin_geom,Point+(" + latlon + "),50000,meters)").json()
    
    lat, lon, mag, time = ([] for i in range(4)) #empty lists for iterating
    #Following pulls quake data for plotting
    for a in range(len(quakejson['features'])):
        lon.append(quakejson['features'][a]['geometry']['coordinates'][0])
        lat.append(quakejson['features'][a]['geometry']['coordinates'][1])
        mag.append(quakejson['features'][a]['properties']['magnitude'])
        time.append(UTCDateTime(quakejson['features'][a]['properties']['origintime']))
    #Dataframe of quake info
    df = pd.DataFrame({"lat":lat, "lon":lon, "time":time,"mag":mag})
    #sort dataframe by origin time
    df = df.sort_values(by="time")
    df = df.dropna()
    #Initiate plot
    fig, ax = plt.subplots(figsize = (10, 10), dpi = 100)
    ax=fig.add_axes([0,0,1,1]) #removing whitespace
    #Basemap plot - world topo map can be subbed for preferred map
    m = Basemap(projection = "merc", llcrnrlon = lon[0] - .35, llcrnrlat =  lat[0] - .35,
              urcrnrlon =  lon[0] + .35, urcrnrlat = lat[0] + .35, epsg = 2193, fix_aspect=False)
    m.arcgisimage(service = "World_Topo_Map", xpixels = 1000, ypixels = 1000, dpi = 1000)
    
    #timelist creates a timestamp for every minute, up to a defined user limit.
    #this is used to create a frame for every minute, showing all quakes prior to said stamp
    timelist = [list(df['time'])[0] + 60 * x for x in range(minutes)]
    #Strings of above to plot as annotation
    stime = [str(x) for x in timelist]
    
    #Defining initial blank plot to offset to
    scat = m.scatter([],[], alpha = 0.6, edgecolor= "black", linewidths = 0.6, c= 'crimson')  
    
    #messy part defining size of legend elements, probably should be loop
    l2 = plt.scatter([-1000],[-1000], s = 2**3.2*2, edgecolors='none', c= 'crimson')
    l3 = plt.scatter([-1000],[-1000], s = 3**3.2*2, edgecolors='none', c= 'crimson')
    l4 = plt.scatter([-1000],[-1000], s = 4**3.2*2, edgecolors='none', c= 'crimson')
    l5 = plt.scatter([-1000],[-1000], s = 5**3.2*2, edgecolors='none', c= 'crimson')
    l6 = plt.scatter([-1000],[-1000], s = 6**3.2*2, edgecolors='none', c= 'crimson')
    l7 = plt.scatter([-1000],[-1000], s = 7**3.2*2, edgecolors='none', c= 'crimson')
    l8 = plt.scatter([-1000],[-1000], s = 8**3.2*2, edgecolors='none', c= 'crimson')
    #legend labels 
    labels = [str(x + 1) + "-" + str(x + 2) for x in range(7)]
    plt.legend([l2,l3,l4,l5,l6,l7, l8], labels, title = "Magnitude", loc = 'lower center', 
               labelspacing = 1, ncol = 7, borderpad = 1.7, columnspacing = 2.3, handletextpad = 1.2)
   
    #annotations - first one is the timestamp, second one is a count of the earthquakes
    annotation = ax.annotate(str(stime[0]), xy=(625, 950), xycoords='axes pixels', 
                             fontsize=14, c ='black')
    annotationcount = ax.annotate("Earthquake count: 0", xy = (20, 950), xycoords='axes pixels', 
                                  fontsize = 17, c = 'black')
    annotation.set_animated(True)
    annotationcount.set_animated(True) 
    ax.set_aspect('auto')
    
    #"initial" blank function, negative values help hide "initial" circle
    def init():
        scat.set_offsets([[],[]])
        return scat, annotation, annotationcount
    
    #animation function - plots all quakes before minute "a", including updating the annotations
    def update(a):
        df2 = df[df['time'] < timelist[a]]
        x, y = m(list(df2['lon']), list(df2['lat']))
        scat.set_offsets(np.c_[x, y])
        scat._sizes=[(int(str(x)[0])+1)**3.2 * 2 for x in list(df2['mag'])]
        annotation.set_text(stime[a])
        annotationcount.set_text("Earthquake count: " + str(len(df2)))
        return scat,
    #function to write animation to an mp4, with the defined filename (automatically as an mp4)
    ani = animation.FuncAnimation(fig, update, init_func=init, blit=False, frames = len(timelist))
    writer = FFMpegWriter(fps=10, bitrate=1800) #change fps if desired
    ani.save((filename + ".mp4"), writer=writer)

#### Function called using variables defined above

In [ ]:
aftershock_animation(firstquakeid, length, filename)